In [1]:
# 거리 계산 함수
from geopy.distance import geodesic

def calculate_distance(lat1, long1, lat2, long2):
    return geodesic((lat1, long1), (lat2, long2)).km

In [2]:
# 무게와 거리를 고려한 단가 계산 함수 
import math

def calculate_shipping_cost(weight_class, distance):
    # 가격 테이블 (단위: 만 원)
    price_table = [
        [4, 5, 7, 8, 11, 13, 18, 20],  # 20km
        [5, 6, 8, 9, 12, 14, 19, 21],  # 30km
        [6, 7, 9, 10, 13, 15, 20, 22], # 50km
        [7, 8, 10, 11, 14, 17, 21, 23], # 70km
        [8, 9, 11, 12, 16, 19, 22, 25], # 90km
        [9, 10, 12, 13, 17, 20, 24, 27], # 110km
        [10, 11, 13, 14, 18, 21, 25, 28], # 130km
        [11, 12, 14, 15, 19, 23, 27, 30], # 150km
        [12, 13, 15, 16, 20, 25, 28, 31], # 170km
        [13, 14, 18, 19, 23, 27, 30, 32], # 200km
        [15, 16, 19, 20, 25, 33, 39, 42], # 250km
        [17, 18, 25, 26, 30, 37, 45, 48], # 300km
        [19, 20, 27, 28, 33, 38, 50, 53], # 350km
        [20, 21, 28, 30, 35, 42, 55, 58], # 400km
        [22, 23, 30, 32, 37, 44, 58, 62], # 450km
        [25, 26, 32, 34, 38, 46, 60, 64], # 500km
        [27, 28, 35, 37, 41, 48, 63, 67], # 550km
        [30, 31, 38, 40, 43, 50, 65, 70]  # 600km
    ]
    
    # 거리와 화물 무게에 대한 인덱스 계산
    distance_index = {
        20: 0, 30: 1, 50: 2, 70: 3, 90: 4,
        110: 5, 130: 6, 150: 7, 170: 8,
        200: 9, 250: 10, 300: 11, 350: 12,
        400: 13, 450: 14, 500: 15, 550: 16, 600: 17
    }
    
    # 화물 무게 클래스를 인덱스로 변환
    weight_index = {
        1: 0, 1.4: 1, 2.5: 2, 3.5: 3, 5: 4, 11: 5, 18: 6, 25: 7
    }
    
    # 거리와 화물 무게의 유효한 인덱스 찾기
    if distance not in distance_index:
        next_distance = min([d for d in distance_index.keys() if d > distance], default=None)
        if next_distance is not None:
            distance = next_distance
    
    if weight_class not in weight_index:
        next_weight = min([w for w in weight_index.keys() if w > weight_class], default=None)
        if next_weight is not None:
            weight_class = next_weight
            
    # 인덱스 얻기
    distance_index_value = distance_index.get(distance, len(price_table) - 1)
    weight_index_value = weight_index.get(weight_class, len(price_table[0]) - 1)
    
    # 가격 계산
    price = price_table[distance_index_value][weight_index_value]
    
    return price

# TEST example 
# weight_class = 25  # 화물 무게 (유효하지 않은 경우)
# distance = 700      # 거리 (유효하지 않은 경우)
# cost = calculate_shipping_cost(weight_class, distance)
# print(f"운송 비용: {cost} 만원")

In [3]:
def normalize(value, min_value, max_value):
    return (value - min_value) / (max_value - min_value)

### 기사의 적합도 계산 함수
-> 차고지 근처로 가는 1차 물류 매칭

In [4]:
def calculate_match_score(driver, logistics):
    # 1. 물류가 기사의 수용 무게 범위에 맞는지 확인
    if logistics['weight'] < driver['min_weight'] or logistics['weight'] > driver['max_weight']:
        return 0  # 물류가 적합하지 않으면 0점
    
    # 2. 기사의 출발지 ~ 물류 창고 간의 거리 계산
    distance_to_warehouse = calculate_distance(driver['departure_lat'], driver['departure_long'], logistics['departure_lat'], logistics['departure_long'])
    
    if distance_to_warehouse > 30: 
        return 0
    
    # 3. 물류 도착지와 기사의 차고지 간의 거리 계산
    distance_to_destination = calculate_distance(logistics['destination_lat'], logistics['destination_long'], driver['destination_lat'], driver['destination_long'])
    
    if distance_to_destination > 30: 
        return 0
    
    # 4. 물류 운송 거리 계산 
    distance_to_logistics = calculate_distance(logistics['departure_lat'], logistics['departure_long'], logistics['destination_lat'], logistics['destination_long'])
    
    # 5. 물류 가격 계산 (물류의 무게와 거리 기반)
    total_distance = distance_to_warehouse + distance_to_destination
    price = calculate_shipping_cost(logistics['weight'], distance_to_logistics)
    
    # 거리와 가격 정규화
    normalized_distance = normalize(total_distance, 2, 400) 
    normalized_price = normalize(price, 4, 70)   # 4만원~70만원 범위에서의 정규화 
    
    # 기사의 선호도에 따른 가중치 조합
    distance_weight = (driver['preference'] / 10) * 1.5
    price_weight = (1 - distance_weight + 0.1) * 0.5
    
    score = (distance_weight * (1 - normalized_distance)) + (price_weight * normalized_price)
    
    return score, distance_to_warehouse, distance_to_destination

In [5]:
import pandas as pd 

input_df = pd.read_excel('INPUT.xlsx')  # 기사 데이터
product_df = pd.read_excel('Dataset/PRODUCT.xlsx')  # 물류 데이터

# 모든 물류에 대해 기사와의 적합도 계산 및 추천
def recommend_top_logistics(driver, product_df, top_n):
    scores = []
    for _, logistics in product_df.iterrows():
        result = calculate_match_score(driver, logistics)
        if result:  # 튜플이 올바르게 반환된 경우 처리
            score, distance_to_warehouse, distance_to_destination = result
            scores.append((logistics['product_id'], score, distance_to_warehouse, distance_to_destination))
    
    # 상위 top_n개의 적합도 높은 물류 추천
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)[:top_n]
    
    # 추천 물류와 점수 및 거리 정보 리스트
    recommended_logistics = []
    for product_id, score, distance_to_warehouse, distance_to_destination in sorted_scores:
        logistics = product_df[product_df['product_id'] == product_id].iloc[0]
        recommended_logistics.append((logistics, score, distance_to_warehouse, distance_to_destination))
    
    return recommended_logistics

# 한 기사에 대해 추천 물류 찾기
driver = input_df.iloc[20]
recommended_logistics = recommend_top_logistics(driver, product_df, 1)

# 기사 정보 출력
print("기사 정보:")
print(driver)

# 1차 물류 추천 결과 및 상위 3개 물류의 score 출력
print("1차 물류 추천 결과:", recommended_logistics)
print("\n1차 물류 추천 결과:")
for logistics, score, distance_to_warehouse, distance_to_destination in recommended_logistics:
    print(f"물류 ID: {logistics['product_id']}, 점수: {score:.2f}, 창고 거리: {distance_to_warehouse:.2f} km, 목적지 거리: {distance_to_destination:.2f} km")

기사 정보:
user_id                                313
max_weight                              14
min_weight                               6
departure           인천광역시 서구 정서진로 47 (오류동)
departure_lat                    37.571656
departure_long                  126.716622
destination           전라남도 강진군 성전면 송계로 764
destination_lat                  34.686319
destination_long                 126.72048
preference                               4
Name: 20, dtype: object
1차 물류 추천 결과: [(user_id                                             64
product_id                                        6330
departure           인천광역시 중구 축항대로165번길 20, 물류센터 (항동7가)
departure_lat                                37.455177
departure_long                              126.616158
name                                     건과제품, 빙과제품_29
destination                      전라남도 해남군 해남읍 땅끝대로 141
destination_lat                              34.556405
destination_long                            126.583106
weight                       

In [8]:
import pandas as pd

# 모든 기사에 대해 물류 추천 및 저장
def recommend_logistics_for_all_drivers(input_df, product_df, top_n, batch_size=1000):
    results = []  # 추천 결과 저장
    batch_count = 0  # 현재 배치 번호
    total_results = 0  # 총 결과 수
    
    for idx, driver in input_df.iterrows():
        # 각 기사에 대해 추천
        recommended_logistics = recommend_top_logistics(driver, product_df, top_n)
        
        # 추천 결과 처리
        for rank, (logistics, score, distance_to_warehouse, distance_to_destination) in enumerate(recommended_logistics, start=1):
            results.append({
                'user_id': driver['user_id'],                     # 기사 ID
                'departure': driver['departure'],                 # 기사 출발지
                'destination': driver['destination'],             # 기사 목적지
                'preference': driver['preference'],               # 기사 선호도
                'result_number': rank,                            # 추천 순위 (1, 2, 3)
                'product_id': logistics['product_id'],            # 물류 ID
                'prod_departure': logistics['departure'],         # 물류 출발지
                'prod_destination': logistics['destination'],     # 물류 목적지
                'score': score,                                   # 적합도 점수
                'distance_to_warehouse': distance_to_warehouse,   # 기사 출발지와 창고 간 거리
                'distance_to_destination': distance_to_destination  # 기사 목적지와 물류 도착지 간 거리
            })
            total_results += 1
        
        print(f"기사 {idx + 1}/{len(input_df)} (user_id: {driver['user_id']}) 추천 완료.")
        
        # 중간 결과 저장
        if total_results % batch_size == 0:
            batch_count += 1
            batch_file = f"First_Route_Part_{batch_count}.xlsx"
            pd.DataFrame(results).to_excel(batch_file, index=False)
            print(f"중간 결과 저장: {batch_file}")
            results = []  # 저장 후 리스트 초기화

    # 마지막 남은 결과 저장
    if results:
        batch_count += 1
        batch_file = f"First_Route_Part_{batch_count}.xlsx"
        pd.DataFrame(results).to_excel(batch_file, index=False)
        print(f"최종 결과 저장: {batch_file}")

# 추천 결과 계산
top_n = 3  # Top 3 추천
batch_size = 1000  # 중간 저장 기준
result_df = recommend_logistics_for_all_drivers(input_df, product_df, top_n, batch_size)

기사 1/10000 (user_id: 412) 추천 완료.
기사 2/10000 (user_id: 399) 추천 완료.
기사 3/10000 (user_id: 426) 추천 완료.
기사 4/10000 (user_id: 432) 추천 완료.
기사 5/10000 (user_id: 297) 추천 완료.
기사 6/10000 (user_id: 419) 추천 완료.
기사 7/10000 (user_id: 304) 추천 완료.
기사 8/10000 (user_id: 272) 추천 완료.
기사 9/10000 (user_id: 290) 추천 완료.
기사 10/10000 (user_id: 180) 추천 완료.
기사 11/10000 (user_id: 192) 추천 완료.
기사 12/10000 (user_id: 453) 추천 완료.
기사 13/10000 (user_id: 183) 추천 완료.
기사 14/10000 (user_id: 372) 추천 완료.
기사 15/10000 (user_id: 305) 추천 완료.
기사 16/10000 (user_id: 285) 추천 완료.
기사 17/10000 (user_id: 242) 추천 완료.
기사 18/10000 (user_id: 409) 추천 완료.
기사 19/10000 (user_id: 395) 추천 완료.
기사 20/10000 (user_id: 303) 추천 완료.
기사 21/10000 (user_id: 313) 추천 완료.
기사 22/10000 (user_id: 458) 추천 완료.
기사 23/10000 (user_id: 198) 추천 완료.
기사 24/10000 (user_id: 223) 추천 완료.
기사 25/10000 (user_id: 218) 추천 완료.
기사 26/10000 (user_id: 258) 추천 완료.
기사 27/10000 (user_id: 286) 추천 완료.
기사 28/10000 (user_id: 173) 추천 완료.
기사 29/10000 (user_id: 188) 추천 완료.
기사 30/10000 (user_id: 3

In [10]:
import pandas as pd

def merge_excel_files_in_order(output_filename, total_parts):
    merged_data = []  # 데이터를 누적할 리스트

    for part_num in range(1, total_parts + 1):
        file_name = f"First_Route_Part_{part_num}.xlsx"
        print(f"병합 중: {file_name}")
        
        try:
            df = pd.read_excel(file_name)  # 각 파일 읽기
            merged_data.append(df)        # 데이터프레임 리스트에 추가
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {file_name}")
            continue

    # 모든 데이터프레임 병합
    final_df = pd.concat(merged_data, ignore_index=True)

    # 병합된 데이터 저장
    final_df.to_excel(output_filename, index=False)
    print(f"모든 파일이 병합되어 저장되었습니다: {output_filename}")

# 병합 실행
merge_excel_files_in_order("First_Route.xlsx", 11)  # 11개 파일 병합

병합 중: First_Route_Part_1.xlsx
병합 중: First_Route_Part_2.xlsx
병합 중: First_Route_Part_3.xlsx
병합 중: First_Route_Part_4.xlsx
병합 중: First_Route_Part_5.xlsx
병합 중: First_Route_Part_6.xlsx
병합 중: First_Route_Part_7.xlsx
병합 중: First_Route_Part_8.xlsx
병합 중: First_Route_Part_9.xlsx
병합 중: First_Route_Part_10.xlsx
병합 중: First_Route_Part_11.xlsx
모든 파일이 병합되어 저장되었습니다: First_Route.xlsx


In [11]:
first_route_df = pd.read_excel('First_Route.xlsx') 
first_route_df = first_route_df[~first_route_df['destination'].str.contains("제주")]
first_route_df.to_excel('First_Route.xlsx', index=False)

### 1차 매칭 물류 사이의 경유지 찾기

In [6]:
from math import sqrt

def point_line_distance(x0, y0, x1, y1, x2, y2):
    """
    주어진 직선과 점 사이의 수직 거리를 계산
    - (x1, y1): 직선의 시작점
    - (x2, y2): 직선의 끝점
    - (x0, y0): 검사할 점
    """
    numerator = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
    denominator = sqrt((y2 - y1)**2 + (x2 - x1)**2)
    return numerator / denominator

def is_same_direction(x1, y1, x2, y2, x0_start, y0_start, x0_end, y0_end):
    """
    주어진 두 경로가 동일한 방향으로 진행하는지 확인합니다.
    x1, y1: 1차 물류 출발지
    x2, y2: 1차 물류 도착지
    x0_start, y0_start: 경유 물류 출발지
    x0_end, y0_end: 경유 물류 도착지
    """
    # 1차 물류의 진행 방향 (출발지 -> 도착지)
    main_lat_diff = x2 - x1
    main_long_diff = y2 - y1

    # 경유 물류의 진행 방향 (출발지 -> 도착지)
    inter_lat_diff = x0_end - x0_start
    inter_long_diff = y0_end - y0_start

    # 진행 방향 비교: 경로의 위도/경도 변화가 동일한지 확인
    return (
        (main_lat_diff * inter_lat_diff >= 0) and
        (main_long_diff * inter_long_diff >= 0)
    )

In [7]:
# 중간 경유지 물류 추천 함수
def find_intermediate_logistics(main_route, product_df, radius):
    intermediate_recommendations = []

    # main_route의 출발지와 도착지 좌표
    x1, y1 = main_route['departure_lat'], main_route['departure_long']
    x2, y2 = main_route['destination_lat'], main_route['destination_long']
    total_main_route_distance = calculate_distance(x1, y1, x2, y2)

    for _, logistics in product_df.iterrows():
        # 물류의 창고 위치 찾기
        x0_start, y0_start = logistics['departure_lat'], logistics['departure_long']
        x0_end, y0_end = logistics['destination_lat'], logistics['destination_long']

        # 출발지와 도착지 모두 직선 반경 내에 있는지 확인
        start_distance = point_line_distance(x0_start, y0_start, x1, y1, x2, y2)
        end_distance = point_line_distance(x0_end, y0_end, x1, y1, x2, y2)

        end_distance_c = calculate_distance(x0_end, y0_end, x2, y2)
        distance_ratio = end_distance_c / total_main_route_distance  # 중간 경유지 위주 추천 

        if start_distance <= radius and end_distance <= radius and 0.4 <= distance_ratio <= 0.7:
            # 방향이 동일한지 확인
            if is_same_direction(x1, y1, x2, y2, x0_start, y0_start, x0_end, y0_end):
                distance_to_logistics = calculate_distance(x0_start, y0_start, x0_end, y0_end)  # 물류 이동 거리...
                distance_to_warehouse = calculate_distance(x1, y1, x0_start, y0_start)
                distance_to_dest = calculate_distance(x0_end, y0_end, x2, y2)

                price = calculate_shipping_cost(logistics['weight'], distance_to_logistics)
                total_distance = distance_to_logistics + distance_to_dest + distance_to_warehouse

                normalized_distance = normalize(total_distance, 10, 400)
                normalized_price = normalize(price, 4, 70)

                distance_weight = 0.8
                price_weight = 1 - distance_weight

                score = (distance_weight * (1 - normalized_distance)) + (price_weight * normalized_price)

                # 추천 결과에 추가
                intermediate_recommendations.append({
                    "logistics": logistics,
                    "score": score,
                    "price": price,
                    "distance_to_warehouse": distance_to_warehouse,
                    "distance_to_dest": distance_to_dest,
                })

    # 추천 결과 중 score 기준 상위 n개만 선택
    n = 1
    intermediate_recommendations = sorted(
        intermediate_recommendations, key=lambda l: l['score'], reverse=True
    )[:n]

    return intermediate_recommendations

# 첫 번째 1차 추천 물류에 대해 경유지 물류 찾기
for main_logistics in recommended_logistics:
    logistics_series, score, distance_to_warehouse, distance_to_destination = main_logistics

    # Pandas Series를 dict처럼 활용
    intermediate_logistics = find_intermediate_logistics(logistics_series, product_df, 30)

    print(f"\n경유지 물류 추천 결과 for {logistics_series['product_id']} (1차 물류):")
    for item in intermediate_logistics:
        logistics = item["logistics"]  # item에서 logistics 추출
        score = item["score"]  # 점수 추출
        price = item["price"]
        distance_to_warehouse = item["distance_to_warehouse"]
        distance_to_dest = item["distance_to_dest"]

        # 물류 정보 출력
        print(f"  - 물류 ID: {logistics['product_id']}, 출발지: {logistics['departure']}, "
              f"도착지: {logistics['destination']}, 가격: {price}, 점수: {score:.2f}, "
              f"창고 거리: {distance_to_warehouse:.2f} km, 목적지 거리: {distance_to_dest:.2f} km")



경유지 물류 추천 결과 for 6330 (1차 물류):
  - 물류 ID: 9810, 출발지: 경기도 시흥시 만해로 43, 외 3필지 (정왕동), 도착지: 전라북도 군산시 서해로 194 (소룡동, 군산항제5부두), 가격: 31, 점수: 0.23, 창고 거리: 17.95 km, 목적지 거리: 157.20 km


In [12]:
import pandas as pd

# 1차 경로 정보 로드
first_route_df = pd.read_excel('First_Route.xlsx')  # 1차 경로 데이터
product_df = pd.read_excel('Dataset/PRODUCT.xlsx')  # 물류 데이터
warehouse_df = pd.read_excel('Dataset/WAREHOUSE.xlsx')  # 창고 데이터

stopover_results = []

def find_intermediate_logistics(main_route, product_df, warehouse_df, radius):
    intermediate_recommendations = []
    
    # main_route의 출발지와 도착지 좌표
    product_location = product_df[product_df['product_id'] == main_route['product_id']].iloc[0]
    x1, y1 = product_location['departure_lat'], product_location['departure_long']
    x2, y2 = product_location['destination_lat'], product_location['destination_long']
    total_main_route_distance = calculate_distance(x1, y1, x2, y2)
    
    if total_main_route_distance == 0:
        print(f"Warning: total_main_route_distance is 0 for product_id {main_route['product_id']}. Skipping this route.")
        return intermediate_recommendations  # 이 row는 건너뛰고 빈 결과 반환
    
    for _, logistics in product_df.iterrows():
        # 물류의 창고 위치 찾기
        x0_start, y0_start = logistics['departure_lat'], logistics['departure_long']
        x0_end, y0_end = logistics['destination_lat'], logistics['destination_long']
        
        # 출발지와 도착지 모두 직선 반경 내에 있는지 확인
        start_distance = point_line_distance(x0_start, y0_start, x1, y1, x2, y2)
        end_distance = point_line_distance(x0_end, y0_end, x1, y1, x2, y2)

        end_distance_c = calculate_distance(x0_end, y0_end, x2, y2)
        distance_ratio = end_distance_c / total_main_route_distance   # 중간쪽 경유지 위주 추천 
        
        if start_distance <= radius and end_distance <= radius and 0.4 <= distance_ratio <= 0.7:
            # 방향이 동일한지 확인
            if is_same_direction(x1, y1, x2, y2, x0_start, y0_start, x0_end, y0_end):
                distance_to_logistics = calculate_distance(x0_start, y0_start, x0_end, y0_end)  # 물류 이동 거리...
                distance_to_warehouse = calculate_distance(x1, y1, x0_start, y0_start)
                distance_to_dest = calculate_distance(x0_end, y0_end, x2, y2)
                
                price = calculate_shipping_cost(logistics['weight'], distance_to_logistics)
                total_distance = distance_to_logistics + distance_to_dest + distance_to_warehouse
                
                normalized_distance = normalize(total_distance, 10, 400)
                normalized_price = normalize(price, 4, 70)
                
                distance_weight = 0.8
                price_weight = 1 - distance_weight
                
                score = (distance_weight * (1 - normalized_distance)) + (price_weight * normalized_price)
                
                # 추천 결과에 추가
                intermediate_recommendations.append({
                    "logistics": logistics,
                    "score": score,
                    "price": price,
                    "distance_to_warehouse": distance_to_warehouse,
                    "distance_to_dest": distance_to_dest,
                })
                
    # 추천 결과 중 score 기준 상위 n개만 선택
    n = 1
    intermediate_recommendations = sorted(
        intermediate_recommendations, key=lambda l: l['score'], reverse=True
    )[:n]
    
    return intermediate_recommendations

# 모든 1차 경로에 대해 경유지 추천 수행
for idx, main_route in first_route_df.iterrows():
    intermediate_logistics = find_intermediate_logistics(main_route, product_df, warehouse_df, 30)
    for item in intermediate_logistics:
        logistics = item["logistics"]
        score = item["score"]
        price = item["price"]
        distance_to_warehouse = item["distance_to_warehouse"]
        distance_to_dest = item["distance_to_dest"]
        
        stopover_results.append({
            "main_product_id": main_route["product_id"],
            "main_departure": main_route["departure"],
            "main_destination": main_route["destination"],
            "prod_departure": main_route["prod_departure"],
            "prod_destination": main_route["prod_destination"],
            "stopover_product_id": logistics["product_id"],
            "stopover_departure": logistics["departure"],
            "stopover_destination": logistics["destination"],
            "distance_to_warehouse": distance_to_warehouse,
            "distance_to_dest": distance_to_dest,
            "score": score,
            "price": price
        })

    print(f"진행 상황: {idx + 1} / {len(first_route_df)} - 1차 물류 {main_route['product_id']}의 경유지 추천 완료.")

# 결과를 엑셀 파일로 저장
stopover_df = pd.DataFrame(stopover_results)
stopover_df.to_excel('First_Stopover.xlsx', index=False)
print("\n모든 경유지 추천 결과가 First_Stopover.xlsx 파일로 저장되었습니다.")

진행 상황: 1 / 23786 - 1차 물류 746의 경유지 추천 완료.
진행 상황: 2 / 23786 - 1차 물류 3010의 경유지 추천 완료.
진행 상황: 3 / 23786 - 1차 물류 720의 경유지 추천 완료.
진행 상황: 4 / 23786 - 1차 물류 14606의 경유지 추천 완료.
진행 상황: 5 / 23786 - 1차 물류 14974의 경유지 추천 완료.
진행 상황: 6 / 23786 - 1차 물류 10076의 경유지 추천 완료.
진행 상황: 7 / 23786 - 1차 물류 10239의 경유지 추천 완료.
진행 상황: 8 / 23786 - 1차 물류 5547의 경유지 추천 완료.
진행 상황: 9 / 23786 - 1차 물류 14039의 경유지 추천 완료.
진행 상황: 10 / 23786 - 1차 물류 5330의 경유지 추천 완료.
진행 상황: 11 / 23786 - 1차 물류 3443의 경유지 추천 완료.
진행 상황: 12 / 23786 - 1차 물류 12942의 경유지 추천 완료.
진행 상황: 13 / 23786 - 1차 물류 11423의 경유지 추천 완료.
진행 상황: 14 / 23786 - 1차 물류 10652의 경유지 추천 완료.
진행 상황: 15 / 23786 - 1차 물류 13859의 경유지 추천 완료.
진행 상황: 16 / 23786 - 1차 물류 11072의 경유지 추천 완료.
진행 상황: 17 / 23786 - 1차 물류 14207의 경유지 추천 완료.
진행 상황: 18 / 23786 - 1차 물류 14275의 경유지 추천 완료.
진행 상황: 19 / 23786 - 1차 물류 15153의 경유지 추천 완료.
진행 상황: 20 / 23786 - 1차 물류 3235의 경유지 추천 완료.
진행 상황: 21 / 23786 - 1차 물류 6330의 경유지 추천 완료.
진행 상황: 22 / 23786 - 1차 물류 1925의 경유지 추천 완료.
진행 상황: 23 / 23786 - 1차 물류 11544의 경유지 추천 완료.
진행 상황: 2

### 2차 경유지 추천

In [8]:
import pandas as pd

# 데이터 로드
first_stopover_df = pd.read_excel('First_Stopover.xlsx')  # 1차 경유지 데이터
input_df = pd.read_excel('INPUT.xlsx')  # 각 물류의 user_id 정보를 포함한 데이터
product_df = pd.read_excel('Dataset/PRODUCT.xlsx')  # 물류 데이터
warehouse_df = pd.read_excel('Dataset/WAREHOUSE.xlsx')  # 창고 데이터

results = []  # 최종 결과 저장
intermediate_results = []  # 중간 저장용 리스트

def get_lat_long(product_id, product_df, location_type):
    product_row = product_df[product_df["product_id"] == product_id]
    if not product_row.empty:
        if location_type == "departure":
            return product_row.iloc[0]["departure_lat"], product_row.iloc[0]["departure_long"]
        elif location_type == "destination":
            return product_row.iloc[0]["destination_lat"], product_row.iloc[0]["destination_long"]
    return None, None  # 값이 없을 경우


def find_secondary_stop(main_route, first_stop, product_df, warehouse_df, radius, n=1):
    secondary_recommendations = []

    # 1차 경유지 도착지를 새로운 출발지로 설정
    first_stop_end_lat = first_stop["stopover_destination_lat"]
    first_stop_end_long = first_stop["stopover_destination_long"]

    # 원래 1차 물류의 최종 목적지
    main_route_dest_lat = main_route["main_destination_lat"]
    main_route_dest_long = main_route["main_destination_long"]

    for _, logistics in product_df.iterrows():
        # 물류의 창고 위치 찾기
        warehouse_location = warehouse_df[warehouse_df["USER_ID"] == logistics["user_id"]].iloc[0]
        x0_start, y0_start = warehouse_location["location_lat"], warehouse_location["location_long"]
        x0_end, y0_end = logistics["destination_lat"], logistics["destination_long"]

        # 출발지가 1차 경유지 도착지 반경 내에 있어야 함
        start_distance = calculate_distance(first_stop_end_lat, first_stop_end_long, x0_start, y0_start)
        end_distance = calculate_distance(x0_end, y0_end, main_route_dest_lat, main_route_dest_long)

        if start_distance <= radius and end_distance <= radius:
            # 방향이 일치하는지 확인
            if is_same_direction(
                first_stop_end_lat, first_stop_end_long,
                main_route_dest_lat, main_route_dest_long,
                x0_start, y0_start, x0_end, y0_end
            ):
                # 거리와 가격 계산
                distance_to_logistics = calculate_distance(x0_start, y0_start, x0_end, y0_end)
                price = calculate_shipping_cost(logistics["weight"], distance_to_logistics)

                normalized_distance = normalize(distance_to_logistics, 10, 400)
                normalized_price = normalize(price, 4, 70)

                # 기사의 선호도에 따른 가중치 조합
                distance_weight = 0.7
                price_weight = 1 - distance_weight

                score = (distance_weight * (1 - normalized_distance)) + (price_weight * normalized_price)

                # 추천 결과에 추가
                secondary_recommendations.append({
                    "logistics": logistics,
                    "score": score,
                    "price": price
                })

    # 상위 n개의 경유지 선택
    secondary_recommendations = secondary_recommendations[:n]

    return secondary_recommendations

# 순서대로 1차 경유지 데이터 처리
for idx, first_stop_row in first_stopover_df.iterrows():
    # user_id를 INPUT.xlsx에서 참조
    main_departure = first_stop_row["main_departure"]
    main_destination = first_stop_row["main_destination"]

    user_id_row = input_df[
        (input_df["departure"] == main_departure) &
        (input_df["destination"] == main_destination)
    ]

    if user_id_row.empty:
        print(f"Warning: No matching user_id found for departure={main_departure} and destination={main_destination}. Skipping...")
        continue

    user_id = user_id_row.iloc[0]["user_id"]

    # Main route 설정
    main_product_id = first_stop_row["main_product_id"]
    main_departure_lat, main_departure_long = get_lat_long(main_product_id, product_df, location_type="departure")
    main_destination_lat, main_destination_long = get_lat_long(main_product_id, product_df, location_type="destination")

    main_route = {
        "user_id": user_id,
        "main_product_id": main_product_id,
        "main_departure": main_departure,
        "main_destination": main_destination,
        "main_departure_lat": main_departure_lat,
        "main_departure_long": main_departure_long,
        "main_destination_lat": main_destination_lat,
        "main_destination_long": main_destination_long,
    }

    # 1차 경유지 정보
    stopover_product_id = first_stop_row["stopover_product_id"]
    stopover_departure_lat, stopover_departure_long = get_lat_long(stopover_product_id, product_df, location_type="departure")
    stopover_destination_lat, stopover_destination_long = get_lat_long(stopover_product_id, product_df, location_type="destination")

    first_stop = {
        "stopover_product_id": stopover_product_id,
        "stopover_departure": first_stop_row["stopover_departure"],
        "stopover_destination": first_stop_row["stopover_destination"],
        "stopover_departure_lat": stopover_departure_lat,
        "stopover_departure_long": stopover_departure_long,
        "stopover_destination_lat": stopover_destination_lat,
        "stopover_destination_long": stopover_destination_long,
        "score": first_stop_row["score"],
        "price": first_stop_row["price"],
    }

    # 2차 경유지 추천
    second_stops = find_secondary_stop(main_route, first_stop, product_df, warehouse_df, 50, n=1)
    if not second_stops:
        continue

    second_stop = second_stops[0]

    # 총 점수와 가격 계산
    total_score = first_stop["score"] + second_stop["score"]
    total_price = first_stop["price"] + second_stop["price"]

    # 결과 저장
    result_entry = {
        "user_id": main_route["user_id"],
        "main_product_id": main_route["main_product_id"],
        "main_departure": main_route["main_departure"],
        "main_destination": main_route["main_destination"],
        "stopover_id": first_stop["stopover_product_id"],
        "stopover_departure": first_stop["stopover_departure"],
        "stopover_destination": first_stop["stopover_destination"],
        "second_stop_id": second_stop["logistics"]["product_id"],
        "second_stop_departure": second_stop["logistics"]["departure"],
        "second_stop_destination": second_stop["logistics"]["destination"],
        "score": total_score,
        "price": total_price
    }

    results.append(result_entry)
    intermediate_results.append(result_entry)

    # 중간 저장: 첫 30개 결과 저장
    if len(intermediate_results) == 5:
        pd.DataFrame(intermediate_results).to_excel('Intermediate_30_Secondary_Stopover.xlsx', index=False)
        print(f"중간 결과 저장: Intermediate_30_Secondary_Stopover.xlsx")

    print(f"Processed row {idx + 1} / {len(first_stopover_df)}")

# 모든 결과를 엑셀 파일로 저장
results_df = pd.DataFrame(results)
results_df.to_excel('Final_Secondary_Stopover_Results.xlsx', index=False)
print("\n최종 결과가 Final_Secondary_Stopover_Results.xlsx 파일로 저장되었습니다.")

Processed row 1 / 23680
Processed row 2 / 23680
Processed row 3 / 23680
Processed row 4 / 23680
중간 결과 저장: Intermediate_30_Secondary_Stopover.xlsx
Processed row 5 / 23680
Processed row 6 / 23680
Processed row 7 / 23680
Processed row 8 / 23680
Processed row 9 / 23680
Processed row 10 / 23680
Processed row 16 / 23680
Processed row 17 / 23680
Processed row 18 / 23680
Processed row 22 / 23680
Processed row 23 / 23680
Processed row 24 / 23680
Processed row 28 / 23680
Processed row 29 / 23680
Processed row 30 / 23680
Processed row 31 / 23680
Processed row 32 / 23680
Processed row 33 / 23680
Processed row 34 / 23680
Processed row 35 / 23680
Processed row 36 / 23680
Processed row 40 / 23680
Processed row 41 / 23680
Processed row 42 / 23680
Processed row 45 / 23680
Processed row 46 / 23680
Processed row 48 / 23680
Processed row 50 / 23680
Processed row 51 / 23680
Processed row 52 / 23680
Processed row 53 / 23680
Processed row 54 / 23680
Processed row 55 / 23680
Processed row 56 / 23680
Processed

* 이전 코드 (map으로 시각적 확인용)

In [31]:
import plotly.express as px

def plot_route(main_logistics, first_stop, second_stop, warehouse_df):
    # 창고 위치에서 출발지 구하기
    warehouse_location = warehouse_df[warehouse_df["USER_ID"] == main_logistics["user_id"]].iloc[0]
    x0_start, y0_start = warehouse_location["location_lat"], warehouse_location["location_long"]
    
    routes = [
        {"lat": x0_start, "lon": y0_start, "name": "Main Start"},  # 출발지
        {"lat": main_logistics["destination_lat"], "lon": main_logistics["destination_long"], "name": "Main End"},  # 목적지
        {"lat": first_stop["logistics"]["destination_lat"], "lon": first_stop["logistics"]["destination_long"], "name": "First Stop"},
        {"lat": second_stop["logistics"]["destination_lat"], "lon": second_stop["logistics"]["destination_long"], "name": "Second Stop"},
    ]
    
    # 지도 시각화
    fig = px.scatter_mapbox(
        routes,
        lat="lat",
        lon="lon",
        hover_name="name",
        mapbox_style="open-street-map",
        zoom=5
    )
    fig.show()

In [32]:
def find_secondary_stop(main_route, first_stop, product_df, warehouse_df, radius, n=1):
    secondary_recommendations = []

    # 1차 경유지 도착지를 새로운 출발지로 설정
    first_stop_end_lat = first_stop["logistics"]["destination_lat"]
    first_stop_end_long = first_stop["logistics"]["destination_long"]

    # 원래 1차 물류의 최종 목적지
    main_route_dest_lat = main_route["destination_lat"]
    main_route_dest_long = main_route["destination_long"]

    for _, logistics in product_df.iterrows():
        # 물류의 창고 위치 찾기
        warehouse_location = warehouse_df[warehouse_df["USER_ID"] == logistics["user_id"]].iloc[0]
        x0_start, y0_start = warehouse_location["location_lat"], warehouse_location["location_long"]
        x0_end, y0_end = logistics["destination_lat"], logistics["destination_long"]

        # 출발지가 1차 경유지 도착지 반경 내에 있어야 함
        start_distance = calculate_distance(first_stop_end_lat, first_stop_end_long, x0_start, y0_start)
        end_distance = calculate_distance(x0_end, y0_end, main_route_dest_lat, main_route_dest_long)

        if start_distance <= radius and end_distance <= radius:
            # 방향이 일치하는지 확인
            if is_same_direction(
                first_stop_end_lat, first_stop_end_long,
                main_route_dest_lat, main_route_dest_long,
                x0_start, y0_start, x0_end, y0_end
            ):
                # 거리와 가격 계산
                distance_to_logistics = calculate_distance(x0_start, y0_start, x0_end, y0_end)
                price = calculate_shipping_cost(logistics["weight"], distance_to_logistics)

                normalized_distance = normalize(distance_to_logistics, 10, 400)
                normalized_price = normalize(price, 4, 70)

                # 기사의 선호도에 따른 가중치 조합
                distance_weight = 0.7
                price_weight = 1 - distance_weight

                score = (distance_weight * (1 - normalized_distance)) + (price_weight * normalized_price)

                # 추천 결과에 추가
                secondary_recommendations.append({
                    "logistics": logistics,
                    "score": score,
                    "price": price
                })

    # 상위 n개의 경유지 선택
    secondary_recommendations = sorted(
        secondary_recommendations, key=lambda l: l['score'], reverse=True
    )[:n]

    return secondary_recommendations

# 1차 추천 물류 기준으로 경유지 1~2차 경로 찾기
for main_logistics in recommended_logistics:
    print(f"\n1차 물류 경로: {main_logistics['product_id']} (출발지 -> 목적지)")
    
    # 1차 경유지 찾기
    first_stops = find_intermediate_logistics(main_logistics, product_df, warehouse_df, 50)
    if not first_stops:
        print("  - 1차 경유지 없음")
        continue

    first_stop = first_stops[0]  # 상위 1개 선택
    print(f"  1차 경유지: {first_stop['logistics']['product_id']} ("
          f"{first_stop['logistics']['departure']} -> {first_stop['logistics']['destination']})")

    # 2차 경유지 찾기
    second_stops = find_secondary_stop(main_logistics, first_stop, product_df, warehouse_df, 50)
    if not second_stops:
        print("  - 2차 경유지 없음")
        continue

    second_stop = second_stops[0]  # 상위 1개 선택
    print(f"  2차 경유지: {second_stop['logistics']['product_id']} ("
          f"{second_stop['logistics']['departure']} -> {second_stop['logistics']['destination']})")


1차 물류 경로: 2723 (출발지 -> 목적지)
  1차 경유지: 4317 (경기도 시흥시 만해로 43(정왕동) -> 경상북도 김천시 공단3길 94 (응명동))
  2차 경유지: 3130 (대구광역시 서구 와룡로69길 7, 중리동1053 창고시설(삼화식품(주) 주건출물제1동 (중리동) -> 부산광역시 남구 우암로 165 (우암동, 동국보세장치장))


### 최종 학습 데이터셋 완성

In [26]:
import pandas as pd

# 파일 로드
result_df = pd.read_excel("Temp_Result/Final_Secondary_Stopover_Results.xlsx")  # 결과 파일
product_df = pd.read_excel("Dataset/PRODUCT.xlsx")  # 물류 데이터
first_route_df = pd.read_excel("Temp_Result/First_Route.xlsx")  # 수정된 파일
input_df = pd.read_excel("INPUT.xlsx")  # departure_lat, departure_long을 가져올 파일

def get_lat_long_from_product(product_id, product_df):
    product_row = product_df[product_df["product_id"] == product_id]
    if not product_row.empty:
        departure_lat = product_row.iloc[0]["departure_lat"]
        departure_long = product_row.iloc[0]["departure_long"]
        destination_lat = product_row.iloc[0]["destination_lat"]
        destination_long = product_row.iloc[0]["destination_long"]
        return departure_lat, departure_long, destination_lat, destination_long
    return None, None, None, None  # 값이 없을 경우

def get_user_lat_long(user_id, product_id, first_route_df, input_df):
    # user_id와 product_id가 모두 일치하는 행을 first_route_df에서 찾음
    user_row = first_route_df[(first_route_df["user_id"] == user_id) & (first_route_df["product_id"] == product_id)]
    if not user_row.empty:
        # 입력 파일에서 departure_lat, departure_long 값 가져오기
        user_departure = user_row.iloc[0]["departure"]
        user_departure_lat = input_df[input_df["user_id"] == user_id].iloc[0]["departure_lat"]
        user_departure_long = input_df[input_df["user_id"] == user_id].iloc[0]["departure_long"]
        user_destination = user_row.iloc[0]["destination"]
        user_destination_lat = input_df[input_df["user_id"] == user_id].iloc[0]["destination_lat"]
        user_destination_long = input_df[input_df["user_id"] == user_id].iloc[0]["destination_long"]
        return user_departure, user_departure_lat, user_departure_long, user_destination, user_destination_lat, user_destination_long
    return None, None, None, None, None, None  # 값이 없을 경우

# 위경도 정보를 추가할 리스트 생성
enhanced_data = []

# Result 파일 처리
for _, row in result_df.iterrows():
    user_id = row["user_id"]
    main_product_id = row["main_product_id"]
    
    # 사용자 위치 위경도 (user_id와 main_product_id를 기준으로 매칭)
    user_departure, user_departure_lat, user_departure_long, user_destination, user_destination_lat, user_destination_long = get_user_lat_long(
        user_id, main_product_id, first_route_df, input_df
    )
    
    # Main Route 정보
    main_departure_lat, main_departure_long, main_destination_lat, main_destination_long = get_lat_long_from_product(
        row["main_product_id"], product_df
    )
    
    # First Stopover 정보
    first_departure_lat, first_departure_long, first_destination_lat, first_destination_long = get_lat_long_from_product(
        row["stopover_id"], product_df
    )
    
    # Second Stopover 정보
    second_departure_lat, second_departure_long, second_destination_lat, second_destination_long = get_lat_long_from_product(
        row["second_stop_id"], product_df
    )
    
    # 행 데이터 생성
    enhanced_data.append({
        "user_id": user_id,
        "user_departure": user_departure,
        "user_departure_lat": user_departure_lat,
        "user_departure_long": user_departure_long,
        "user_destination": user_destination,
        "user_destination_lat": user_destination_lat,
        "user_destination_long": user_destination_long,
        "main_product_id": row["main_product_id"],
        "main_departure": row["main_departure"],
        "main_departure_lat": main_departure_lat,
        "main_departure_long": main_departure_long,
        "main_destination": row["main_destination"],
        "main_destination_lat": main_destination_lat,
        "main_destination_long": main_destination_long,
        "first_stop_id": row["stopover_id"],
        "first_stopover_departure": row["stopover_departure"],
        "first_stopover_departure_lat": first_departure_lat,
        "first_stopover_departure_long": first_departure_long,
        "first_stopover_destination": row["stopover_destination"],
        "first_stopover_destination_lat": first_destination_lat,
        "first_stopover_destination_long": first_destination_long,
        "second_stop_id": row["second_stop_id"],
        "second_stopover_departure": row["second_stop_departure"],
        "second_stopover_departure_lat": second_departure_lat,
        "second_stopover_departure_long": second_departure_long,
        "second_stopover_destination": row["second_stop_destination"],
        "second_stopover_destination_lat": second_destination_lat,
        "second_stopover_destination_long": second_destination_long,
        "score": row["score"],
        "price": row["price"],
    })

# 결과를 데이터프레임으로 변환
enhanced_df = pd.DataFrame(enhanced_data)
enhanced_df = enhanced_df.dropna().reset_index(drop=True)


# 새로운 파일로 저장
enhanced_df.to_excel("Result.xlsx", index=False)
print("Result.xlsx 파일이 생성되었습니다.")


Result.xlsx 파일이 생성되었습니다.
